In [6]:
import os
os.environ["OMP_NUM_THREADS"]='16'
import numpy as np
from scipy import linalg
from functools import reduce
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
import tensornetwork as tn
import time,math,gc,itertools
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import svds
from colorsys import hls_to_rgb
from tqdm import tqdm
import opt_einsum
import sys
def show(z):
    if len(z.shape)==1:
        z=z.reshape((1,z.shape[0]))
    r,arg = np.abs(z),np.angle(z)
    h,s = (arg + np.pi)  / (2 * np.pi) + 0.5,0.8
    l=r**(1/2.2) #gamma correction
    l=l/(2*np.max(l))
    c = np.array(np.vectorize(hls_to_rgb) (h,l,s)).swapaxes(0,2).swapaxes(0,1)
    plt.imshow(c)
    plt.show()
MEMORY_LIMIT=30*1024*1024*1024

In [7]:
def getCG(n):
    """shape:[n+1]+[2]*n
    """
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1
    CG=np.array(list(map(lambda x:x/linalg.norm(x),CG.reshape(n+1,-1)))).reshape(CG.shape)
    return CG

assert getCG(3).shape==(4,2,2,2)
CGs=[getCG(i) for i in range(7)]
singlet = np.array([[0,1],[-1,0]])
assert (singlet.dot(singlet)==-np.eye(2)).all()
assert (singlet==-singlet.T).all()

In [8]:
def contract_physical_edges(physical_edges1,physical_edges2):
    """
    Parameters
    ----------
    edges1,edges2:map vertice_id->edge
    """
    for k in physical_edges1:
        if k in physical_edges2:
            physical_edges1[k]^physical_edges2[k]
def contract_disconnected_tensor_network(nodes,edges):
    newNodes=[]
    nodes=set(nodes)
    while len(nodes)>0:
        newSet=set(tn.reachable(next(iter(nodes))))
        newEdges=list(filter(lambda x:x.get_nodes()[0] in newSet or x.get_nodes()[1] in newSet,edges))
        #print(newSet,[[x.get_nodes()[0],x.get_nodes()[1]] for x in newEdges])
        newNode=tn.contractors.auto(newSet,newEdges)
        nodes=nodes.difference(newSet)
        newNodes.append(newNode)
    result=tn.outer_product_final_nodes(newNodes,edges)
    return result 

In [9]:
assert sys.version_info>=(3,7) #will use dict ordering feature
class AKLT_Tensor:
    """
    Attributes
    ----------
    physical_edges:sorted dict vertice_id->edge
    virtual_edges:sorted dict vertice_id->edge
    nodes:sorted dict vertice_id->node or None after SVD
    physical_dimension:int
    virtual_dimension:int
    
    """
    def __init__(self,spins,conns,div):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """        
        self.nodes=dict()
        self.physical_edges=dict()
        self.virtual_edges=dict()
        
        curIndice=[1]*len(spins)
        for vid in div:
            self.nodes[vid]=tn.Node(CGs[spins[vid]])# shape: [n+1]+[2]*n
        for vid1,vid2 in conns:
            if vid1 in div and vid2 in div:
                n1=self.nodes[vid1]
                n2=tn.Node(singlet)
                n3=self.nodes[vid2]

                edges=n1.get_all_edges()
                edges[curIndice[vid1]]=n2[1]
                n1=tn.contract(n1[curIndice[vid1]]^n2[0]).reorder_edges(edges)
                self.nodes[vid1]=n1

                n1[curIndice[vid1]]^n3[curIndice[vid2]]
                curIndice[vid1]+=1
                curIndice[vid2]+=1
        for vid in div:
            n1=self.nodes[vid]
            self.physical_edges[vid]=n1[0]
            num=spins[vid]+1-curIndice[vid]
            if num==0:
                pass
            elif num==1:
                self.virtual_edges[vid]=n1[curIndice[vid]]
            else:
                n2=tn.Node(CGs[num].conj())
                edges=n1.get_all_edges()[0:curIndice[vid]]+[n2[0]]
                for i in range(num):
                    n1[curIndice[vid]+i]^n2[1+i]
                n1=tn.contract_between(n1,n2,output_edge_order=edges)
                self.nodes[vid]=n1
                self.virtual_edges[vid]=edges[-1]
        
        self.nodes=dict(sorted(self.nodes.items()))
        self.physical_edges=dict(sorted(self.physical_edges.items()))
        self.virtual_edges=dict(sorted(self.virtual_edges.items()))
        
        self.physical_dimensions=[e.dimension for e in self.physical_edges.values()]
        self.virtual_dimensions=[e.dimension for e in self.virtual_edges.values()]
        self.physical_dimension=np.prod(self.physical_dimensions)
        self.virtual_dimension=np.prod(self.virtual_dimensions)
        

    
    def print_dimensions(self):
        print("node shapes:",[(i,n.shape) for i,n in self.nodes.items()])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimensions:",self.virtual_dimensions)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def print_topology(self):
        inm={n:i for i,n in self.nodes.items()}
        toPrint=[]
        for n in self.nodes.values():
            for e in n.edges:
                if e in self.physical_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'p',e.dimension])
                elif e in self.virtual_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'v',e.dimension])
                else:
                    toPrint.append([e.node1 and inm[e.node1],e.node2 and inm[e.node2],e.dimension])
        print(toPrint)
    def copy(self):
        node_dict,edge_dict=tn.copy(self.nodes.values())
        rtval=object.__new__(self.__class__)
        
        rtval.nodes={k:node_dict[v] for k,v in self.nodes.items()}
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_edges={k:edge_dict[v] for k,v in self.virtual_edges.items()}
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.virtual_dimensions=self.virtual_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.virtual_dimension=self.virtual_dimension
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            list(t.nodes.values()),
            list(t.physical_edges.values())+list(t.virtual_edges.values())
        ).tensor


In [10]:
class AKLT_Physical_Basis:
    def __init__(self,spins,conns,div,validate=True):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """
        tensor=AKLT_Tensor(spins,conns,div)
        # assert tn.check_correct(tensor.nodes.values())==None wrong for non-connected graph

        
        t1,t2=tensor.copy(),tensor.copy()
        contract_physical_edges(t1.physical_edges,t2.physical_edges)
        mm=contract_disconnected_tensor_network(
            list(t1.nodes.values())+list(t2.nodes.values()),
            list(t1.virtual_edges.values())+list(t2.virtual_edges.values())
        ).tensor
        
        print("begin SVD for ",tensor.virtual_dimension," x ",tensor.virtual_dimension)
        s2,v=linalg.eigh(mm.reshape((tensor.virtual_dimension,tensor.virtual_dimension)))
        s=np.sqrt(s2)
        vh=v.conj().T
        assert np.allclose(v@vh,np.eye(v.shape[0]))
        assert np.allclose(vh@v,np.eye(vh.shape[0]))
        print("maxmin singular values: ",max(s),min(s)," total ",len(s))
        
        #new_tensor=v@np.diag(1/s)
        new_tensor=v@np.diag(1/s)@vh
        new_node=tn.Node(new_tensor.reshape(tensor.virtual_dimensions+[new_tensor.shape[1]]))
        
        for i,e in enumerate(tensor.virtual_edges.values()):
            new_node[i]^e
                
        self.all_nodes=[new_node]+list(tensor.nodes.values())
        self.physical_edges=tensor.physical_edges
        self.virtual_edge=new_node[-1]
        self.virtual_dimension=new_tensor.shape[1]
        self.physical_dimensions=tensor.physical_dimensions.copy()
        self.physical_dimension=tensor.physical_dimension
        self.singular_values=s
        self.v_tensor=v
        
        if validate:
            print('start validate')
            self.test_unitary()
            self.test_idempotent()
            print('validate passed')
        
    def print_dimensions(self):
        print("node shapes:",[n.shape for n in self.all_nodes])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimension:",self.virtual_dimension)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def copy(self):
        node_dict,edge_dict=tn.copy(self.all_nodes)
        rtval=object.__new__(self.__class__)
        
        rtval.all_nodes=[node_dict[v] for v in self.all_nodes]
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_dimension=self.virtual_dimension
        rtval.virtual_edge=edge_dict[self.virtual_edge]
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.singular_values=self.singular_values.copy()
        rtval.v_tensor=self.v_tensor.copy()
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            t.all_nodes,
            list(t.physical_edges.values())+[t.virtual_edge]
        ).tensor
    
    def test_unitary(self):
        a,b=self.copy(),self.copy()
        contract_physical_edges(a.physical_edges,b.physical_edges)
        t=contract_disconnected_tensor_network(
            a.all_nodes+b.all_nodes,
            [a.virtual_edge,b.virtual_edge]
        ).tensor
        assert np.allclose(t,np.eye(self.virtual_dimension))
    def test_idempotent(self):
        def E(v):
            n0=tn.Node(v.reshape(self.physical_dimensions))
            n0_physical_edges={k:n0[i] for i,k in enumerate(self.physical_edges)}
            a,b=self.copy(),self.copy()
            contract_physical_edges(n0_physical_edges,a.physical_edges)
            a.virtual_edge^b.virtual_edge
            nodes=a.all_nodes+b.all_nodes+[n0]
            edges=list(b.physical_edges.values())
            n1=contract_disconnected_tensor_network(nodes,edges)
            return n1.tensor.flatten()
        v=np.random.random(self.physical_dimension)
        assert np.allclose(E(v),E(E(v)))
        
a=AKLT_Physical_Basis([4,2],[[0,1]],[0,1])
b=AKLT_Tensor([4,2],[[0,1]],[0,1])
pd=a.physical_dimension
vd=a.virtual_dimension
ua=a.get_tensor().reshape(pd,-1)@a.v_tensor
sa=np.diag(a.singular_values)
va=a.v_tensor
tb=b.get_tensor().reshape(pd,-1)
assert np.allclose(ua@sa@va.T,tb)
assert np.allclose(va.T@va,np.eye(vd))
assert np.allclose(va@va.T,np.eye(vd))
assert np.allclose(ua.T@ua,np.eye(vd))
assert np.allclose(ua@ua.T,ua@ua.T@ua@ua.T)

a.test_unitary()
a.test_idempotent()


begin SVD for  8  x  8
maxmin singular values:  1.1180339887498951 0.8660254037844386  total  8
start validate
validate passed


In [6]:
# decorated diamond subgraph proj gap
# 0.15830084
# 261it [36:39,  8.43s/it]
# eta0 0.3 UP 2.0 tol 0.0001 eig 3.444978654914206 threshold 3.4
#    gap 0.15326688581022188 ghostgap 0.1467331141897781
# 371it [50:09,  8.11s/it]
# eta0 0.32 UP 2.0 tol 0.0001 eig 3.4111504999632434 threshold 3.36
#     gap 0.16497493903261645 ghostgap 0.15502506096738355

spins=[2,2,2,4, 2,2,2,4, 2,2,2,4, 2,2,2,4, 2,2,2,2,4]
conns=[1,4,2,4,3,4,4,17, 5,8,6,8,7,8,8,18, 
       9,12,10,12,11,12,12,19, 13,16,14,16,15,16,16,20, 
      17,21,18,21,19,21,20,21]
conns=np.array(conns).reshape(-1,2)-1
weights=[.25,.25,.25,.25, .25,.25,.25,.25, 
         .25,.25,.25,.25, .25,.25,.25,.25, 
         1,1,1,1]
proj_parts=[[1,2,3,4,17],[5,6,7,8,18],[9,10,11,12,19],[13,14,15,16,20],[17,18,19,20,21]]
proj_weights=[.25,.25,.25,.25,1]
iso_parts=[[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],
          [17],[18],[19],[20],[21]]
proj_parts=[[i-1 for i in j]for j in proj_parts]
iso_parts=[[i-1 for i in j]for j in iso_parts]




In [7]:
memory_limit=30*1024*1024*1024

proj_U=[AKLT_Physical_Basis(spins,conns,i,validate=False) for i in proj_parts]
iso_U=[AKLT_Physical_Basis(spins,conns,i,validate=False) for i in iso_parts]

virtual_dimensions=[i.virtual_dimension for i in iso_U]
virtual_dimension=np.prod(virtual_dimensions)

def H_proj_single(v,proj_id):
    n0=tn.Node(v.reshape(virtual_dimensions))
    n2=proj_U[proj_id].copy()
    n3=proj_U[proj_id].copy()
    edges=[]
    nodes=[n0]+n2.all_nodes+n3.all_nodes
    
    n2.virtual_edge^n3.virtual_edge
    for i in range(len(iso_parts)):
        if iso_parts[i][0] in proj_parts[proj_id]:
            n1=iso_U[i].copy()
            n4=iso_U[i].copy()
            n0[i]^n1.virtual_edge
            contract_physical_edges(n1.physical_edges,n2.physical_edges)
            contract_physical_edges(n3.physical_edges,n4.physical_edges)
            edges.append(n4.virtual_edge)
            nodes+=n1.all_nodes+n4.all_nodes
        else:
            edges.append(n0[i])
    
    n5=tn.contractors.auto(nodes,edges,memory_limit=memory_limit)
    rtval=n5.tensor.reshape(-1)
    return rtval

def H(v):
    rtval=np.zeros(v.shape)
    for proj_id in range(len(proj_parts)):
        rtval+=proj_weights[proj_id]*H_proj_single(v,proj_id)
        gc.collect(2)
    return rtval
print("virtual_dimensions:",virtual_dimension,virtual_dimensions)
print("--- start timing ---")
start_time = time.time()
v=np.random.random(virtual_dimension)
Hv=H(v)
print("--- %s seconds ---" % (time.time() - start_time))
print('done')

begin SVD for  16  x  16
maxmin singular values:  1.3919410907075054 1.1180339887498951  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3919410907075054 1.1180339887498947  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3919410907075054 1.118033988749895  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3919410907075054 1.1180339887498942  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3919410907075054 1.118033988749895  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3307266185559425 0.9013878188659972  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3307266185559425 0.9013878188659972  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3307266185559425 0.9013878188659972  total  16
begin SVD for  16  x  16
maxmin singular values:  1.3307266185559432 0.9013878188659971  total  16
begin SVD for  3  x  3
maxmin singular values:  1.0 0.9999999999999998  total  3
begin SVD for  3  x  3
maxmin 

In [8]:
tol=0.000001
eta=0.32
UP=np.sum(proj_weights)*1.1

pbar=tqdm()
def OP(v):
    v1=H(v)
    v2=H(v1)
    rtval=-2*v2+(2*eta)*v1+(UP*(UP-eta))*v
    pbar.update(1)
    del v2,v1
    gc.collect(2)
    return rtval

op=LinearOperator(shape=(virtual_dimension,virtual_dimension),matvec=OP)
eigenvalues, eigenvectors = eigsh(op, k=1,tol=tol)
pbar.close()

e=eigenvalues[0]
eig1=0.5*(eta-np.sqrt(2*UP*(UP-eta)-2*e+eta*eta))
eig2=0.5*(eta+np.sqrt(2*UP*(UP-eta)-2*e+eta*eta))
slope=(0.5*(eta+np.sqrt(2*UP*(UP-eta)-2*(e+0.001)+eta*eta))-eig2)/0.001
print('eta0',eta,"UP",UP,'tol',tol,'eig',e,'threshold',UP*(UP-eta))
print('    gap',eig2,'ghostgap',eig1)
slope=(0.5*(eta+np.sqrt(2*UP*(UP-eta)-2*(e+0.0000001)+eta*eta))-eig2)/0.0000001
print(slope*tol*UP)

103it [14:52,  9.06s/it]

KeyboardInterrupt: 

In [ ]:
print(slope*tol*e)


In [ ]:

#eta0 1 UP 2.0 tol 0.01 gap 0.511368284458404 ghostgap 0.488631715541596

#eta0 0.1 UP 2.0 tol 0.01 gap 1.9999999999998295 ghostgap -1.8999999999998294

#3.6027924890033107 vs 3.6
#eigs 0.007243568965032504 0.1927564310349675
#eta0 0.2 UP 2.0 tol 0.01 gap 0.007243568965032504 ghostgap 0.1927564310349675
#eta0 0.3 UP 2.0 tol 0.01 gap 0.17151792050584802 ghostgap 0.12848207949415197


In [ ]:
print('eta0',eta,"UP",UP,'tol',tol,'eig',e,'threshold',UP*(UP-eta))
print('    gap',eig2,'ghostgap',eig1)